# Laboratorio 5

WWI (World Wide Importers) 1 es una empresa encargada de realizar importaciones y venderlas a diferentes clientes en diferentes ciudades de Estados Unidos. Actualmente, la empresa se encuentra buscando servicios de consultoría de BI puesto que desean optimizar sus ganancias, pues consideran que algunos de sus productos no están generando las ganancias que deberían. También, están interesados en saber si hay otros factores que le impiden optimizar sus ganancias. Dado lo anterior, WWI lo contrata a usted para que realice una consultoría de BI, en particular en esta fase, para la creación de la base de datos, la carga de datos y unas consultas iniciales que permitan validar el proceso previo. En esta ocasión quiere que el trabajo lo realicen en grupos de máximo 3 estudiantes, con el fin de comparar dos herramientas que está evaluando para utilizar a futuro en procesos de ETL.
Para esta etapa de la consultoría, la empresa requiere que usted implemente un proceso ETL que le permita extraer los datos de órdenes desde unos archivos CSV y almacenarlos en un modelo dimensional tal que les permita realizar análisis para mejorar entre otros elementos, su eficiencia operativa.

## Análisis exploratorio de los datos

In [1]:
#Instalamos las dependencias necesarias
!pip3 install pandas_profiling
!pip3 install pandas==1.2

Se realizará el análisis exploratorio para cada uno de los datasets proveídos por WWI

In [2]:
# Se importan las librerías y dependencias necesarias para la ejecución del notebook
import numpy as np
import pandas as pd
import matplotlib as plt
from pandas_profiling import ProfileReport

### Variable Dimension City

In [3]:
datos_dim_city = pd.read_csv('datos_originales/dimension_city.csv', delimiter=',')
datos_dim_city.sample(5)

,row ID,City_Key,City,State_Province,Country,Continent,Sales_Territory,Region,Subregion,Latest_Recorded_Population
58,Row58,58,El Paso,Texas,United States,North America,Southwest,Americas,Northern America,649121
75,Row75,75,El Rio,California,United States,North America,Far West,Americas,Northern America,7198
44,Row44,44,El Dorado Springs,Missouri,United States,North America,Plains,Americas,Northern America,3593
95,Row95,95,Austin,Arkansas,United States,North America,Southeast,Americas,Northern America,2038
3,Row3,3,Carrollton,Illinois,United States,North America,Great Lakes,Americas,Northern America,2484


Observamos las primeras filas del dataset

In [4]:
datos_dim_city.head()

,row ID,City_Key,City,State_Province,Country,Continent,Sales_Territory,Region,Subregion,Latest_Recorded_Population
0,Row0,0,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0
1,Row1,1,Carrollton,New York,United States,North America,Mideast,Americas,Northern America,0
2,Row2,2,Carrollton,Virginia,United States,North America,Southeast,Americas,Northern America,4574
3,Row3,3,Carrollton,Illinois,United States,North America,Great Lakes,Americas,Northern America,2484
4,Row4,4,Carrollton,Missouri,United States,North America,Plains,Americas,Northern America,3784


Vemos que la primera fila, con row ID Row0, no aporta información relevante pues todos sus datos están ausentes. Procedemos entonces a eliminarla.

In [5]:
datos_dim_city = datos_dim_city.drop(0)
#Verificamos que se eliminó correctamente
datos_dim_city.head()

,row ID,City_Key,City,State_Province,Country,Continent,Sales_Territory,Region,Subregion,Latest_Recorded_Population
1,Row1,1,Carrollton,New York,United States,North America,Mideast,Americas,Northern America,0
2,Row2,2,Carrollton,Virginia,United States,North America,Southeast,Americas,Northern America,4574
3,Row3,3,Carrollton,Illinois,United States,North America,Great Lakes,Americas,Northern America,2484
4,Row4,4,Carrollton,Missouri,United States,North America,Plains,Americas,Northern America,3784
5,Row5,5,Carrollton,Ohio,United States,North America,Great Lakes,Americas,Northern America,3241


Veamos, ahora, el número de filas y columnas que tenemos en nuestro conjunto de datos:

In [6]:
print("Número de filas o datos: {}".format(len(datos_dim_city)))
print("Número de columnas o variables: {}".format(len(datos_dim_city.columns)))

Número de filas o datos: 97
Número de columnas o variables: 10


Vemos que tenemos 98 datos (filas) y 10 columnas (variables).

Para ver una exploración más exhaustiva, hicimos un pandas profiling de los datos que nos fueron entregados:

In [7]:
profiling = ProfileReport(datos_dim_city)
profiling.to_file(output_file="pandas_profiling/dimension_city_pandas_profiling.html")
profiling

Summarize dataset:   9%|▊         | 2/23 [00:00<00:02,  8.75it/s, Describe variable:df_index]


ImportError: cannot import name 'ABCIndex' from 'pandas.core.dtypes.generic' (c:\Users\Alvaro\anaconda3\lib\site-packages\pandas\core\dtypes\generic.py)

Del anterior reporte podemos concluir y considerar:

- No encontamos filas duplicadas
- Las variables row ID y City_Key tienen un valor distinto en cada fila. Se podría considerar eliminar una de las dos variables.
- No encontramos celdas ausentes.
- Las variables Country, Continent, Region y Subregion tienen el mismo valor en todas las celdas. Esto indica que no aportan información relevante y pueden ser eliminadas.

### Variable Dimension Customer

In [8]:
datos_dim_cust = pd.read_csv('datos_originales/dimension_customer.csv', delimiter=',', encoding='latin-1')
datos_dim_cust.head()

,Customer_Key,Customer,Bill_To_Customer,Category,Buying_Group,Primary_Contact,Postal_Code
0,0,Unknown,NaN,NaN,NaN,NaN,NaN
1,1,Tailspin Toys (Head Office),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Waldemar Fisar,90410.0
2,2,Tailspin Toys (Sylvanite- MT),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Lorena Cindric,90216.0
3,3,Tailspin Toys (Peeples Valley- AZ),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Bhaargav Rambhatla,90205.0
4,4,Tailspin Toys (Medicine Lodge- KS),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Daniel Roman,90152.0


Vemos que la primera fila, con Customer_Key 0, no aporta información relevante pues todos sus datos están ausentes. Procedemos entonces a eliminarla.

In [9]:
datos_dim_cust = datos_dim_cust.drop(0)
#Verificamos que se eliminó correctamente
datos_dim_cust.head()

,Customer_Key,Customer,Bill_To_Customer,Category,Buying_Group,Primary_Contact,Postal_Code
1,1,Tailspin Toys (Head Office),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Waldemar Fisar,90410.0
2,2,Tailspin Toys (Sylvanite- MT),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Lorena Cindric,90216.0
3,3,Tailspin Toys (Peeples Valley- AZ),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Bhaargav Rambhatla,90205.0
4,4,Tailspin Toys (Medicine Lodge- KS),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Daniel Roman,90152.0
5,5,Tailspin Toys (Gasport- NY),Tailspin Toys (Head Office),Novelty Shop,Tailspin Toys,Johanna Huiting,90261.0


Veamos, ahora, el número de filas y columnas que tenemos en nuestro conjunto de datos:

In [10]:
print("Número de filas o datos: {}".format(len(datos_dim_cust)))

print("Número de columnas o variables: {}".format(len(datos_dim_cust.columns)))

Número de filas o datos: 402
Número de columnas o variables: 7


Para ver una exploración más exhaustiva, hicimos un pandas profiling de los datos que nos fueron entregados:

In [11]:
profiling = ProfileReport(datos_dim_cust)
profiling.to_file(output_file="pandas_profiling/dimension_customer_pandas_profiling.html")
profiling

Summarize dataset:  15%|█▌        | 3/20 [00:00<00:00, 22.16it/s, Describe variable:Postal_Code] 


ImportError: cannot import name 'ABCIndex' from 'pandas.core.dtypes.generic' (c:\Users\Alvaro\anaconda3\lib\site-packages\pandas\core\dtypes\generic.py)

Del anterior reporte podemos concluir y considerar:
- No encontamos filas duplicadas
- No encontramos celdas ausentes.
- La Variable Category tiene un único valor. Esto indica que no aporta información relevante y se puede eliminar.
- Las variables Billl_To_Customer y Buying_Group aportan la misma información, por lo que podemos considerar eliminar una de las 2.

### Variable Dimension Date

In [12]:
datos_dim_date = pd.read_csv('datos_originales/dimension_date.csv', delimiter=',', encoding='latin-1')
datos_dim_date.head()

,Date_key,Day_Number,Day_val,Month_val,Short_Month,Calendar_Month_Number,Calendar_Year,Fiscal_Month_Number,Fiscal_Year
0,2013-01-01,1,1,January,Jan,1,2013,3,2013
1,2013-01-02,2,2,January,Jan,1,2013,3,2013
2,2013-01-03,3,3,January,Jan,1,2013,3,2013
3,2013-01-04,4,4,January,Jan,1,2013,3,2013
4,2013-01-05,5,5,January,Jan,1,2013,3,2013


Veamos el número de filas y columnas que se encuentran en nuestro dataset

In [13]:
print("Número de filas o datos: {}".format(len(datos_dim_date)))

print("Número de columnas o variables: {}".format(len(datos_dim_date.columns)))

Número de filas o datos: 1461
Número de columnas o variables: 9


Para ver una exploración más exhaustiva, hicimos un pandas profiling de los datos que nos fueron entregados:

In [14]:
profiling = ProfileReport(datos_dim_date)
profiling.to_file(output_file="pandas_profiling/dimension_date_pandas_profiling.html")
profiling

Summarize dataset:  24%|██▍       | 5/21 [00:00<00:00, 25.13it/s, Describe variable:Fiscal_Year]          


ImportError: cannot import name 'ABCIndex' from 'pandas.core.dtypes.generic' (c:\Users\Alvaro\anaconda3\lib\site-packages\pandas\core\dtypes\generic.py)

Del anterior reporte podemos concluir y considerar:
- No encontamos filas duplicadas
- No encontramos celdas ausentes.
- Las variables Day_Number y Day_val aportan la misma información, por lo que podemos considerar eliminar una de las 2.
- Las variables Month_val y Short_Month aportan la misma información, por lo que podemos considerar eliminar una de las 2.

### Variable Dimension Employee

In [15]:
datos_dim_emp = pd.read_csv('datos_originales/dimension_employee.csv', delimiter=',', encoding='latin-1')
datos_dim_emp.head()

,Employee_Key,Employee,Preferred_Name,Is_Salesperson
0,0,Unknown,NaN,False
1,1,Lily Code,Lily,True
2,2,Isabella Rupp,Isabella,False
3,3,Ethan Onslow,Ethan,False
4,4,Amy Trefl,Amy,True


Vemos que la primera fila, con Employee_Key 0, no aporta información relevante pues todos sus datos están ausentes. Procedemos entonces a eliminarla.

In [16]:
datos_dim_emp = datos_dim_emp.drop(0)
#Verificamos que se eliminó correctamente
datos_dim_emp.head()

,Employee_Key,Employee,Preferred_Name,Is_Salesperson
1,1,Lily Code,Lily,True
2,2,Isabella Rupp,Isabella,False
3,3,Ethan Onslow,Ethan,False
4,4,Amy Trefl,Amy,True
5,5,Jai Shand,Jai,False


Observamos la cantidad de filas y columnas que se encuentran en nuestro dataset

In [17]:
print("Número de filas o datos: {}".format(len(datos_dim_emp)))
print("Número de columnas o variables: {}".format(len(datos_dim_emp.columns)))

Número de filas o datos: 212
Número de columnas o variables: 4


Para hacer una exploración más exhasutiva, hicimos un pandas profiling de los datos

In [18]:
profiling = ProfileReport(datos_dim_emp)
profiling.to_file(output_file="pandas_profiling/dimension_employee_pandas_profiling.html")
profiling

Summarize dataset:  18%|█▊        | 3/17 [00:00<00:00, 36.91it/s, Describe variable:df_index]    


ImportError: cannot import name 'ABCIndex' from 'pandas.core.dtypes.generic' (c:\Users\Alvaro\anaconda3\lib\site-packages\pandas\core\dtypes\generic.py)

Del anterior reporte podemos concluir y considerar:
- No encontamos filas duplicadas
- No encontramos celdas ausentes.
- No se debe realizar ningún paso de preprocesamiento para esta variable

### Variable Dimension Stock Item

In [19]:
datos_dim_stock = pd.read_csv('datos_originales/dimension_stock_item.csv', delimiter=',', encoding='latin-1')
datos_dim_stock.head()

,Stock_Item_Key,Stock_Item,Color,Selling_Package,Buying_Package,Brand,Size_val,Lead_Time_Days,Quantity_Per_Outer,Is_Chiller_Stock,Tax_Rate,Unit_Price,Recommended_Retail_Price,Typical_Weight_Per_Unit
0,0,Unknown,NaN,NaN,NaN,NaN,NaN,0,0,False,",000",",00",",00",",000"
1,1,Void fill 400 L bag (White) 400L,NaN,Each,Each,NaN,400L,14,10,False,"14,000","50,00","74,75","1,000"
2,2,Void fill 300 L bag (White) 300L,NaN,Each,Each,NaN,300L,14,10,False,"14,000","37,50","56,06",",750"
3,3,Void fill 200 L bag (White) 200L,NaN,Each,Each,NaN,200L,14,10,False,"14,000","25,00","37,38",",500"
4,4,Void fill 100 L bag (White) 100L,NaN,Each,Each,NaN,100L,14,10,False,"14,000","12,50","18,69",",250"


Vemos que la primera fila, con Item_Key 0, no aporta información relevante pues todos sus datos están ausentes. Procedemos entonces a eliminarla.

In [20]:
datos_dim_stock = datos_dim_stock.drop(0)
#Verificamos que se eliminó correctamente
datos_dim_stock.head()

,Stock_Item_Key,Stock_Item,Color,Selling_Package,Buying_Package,Brand,Size_val,Lead_Time_Days,Quantity_Per_Outer,Is_Chiller_Stock,Tax_Rate,Unit_Price,Recommended_Retail_Price,Typical_Weight_Per_Unit
1,1,Void fill 400 L bag (White) 400L,NaN,Each,Each,NaN,400L,14,10,False,"14,000","50,00","74,75","1,000"
2,2,Void fill 300 L bag (White) 300L,NaN,Each,Each,NaN,300L,14,10,False,"14,000","37,50","56,06",",750"
3,3,Void fill 200 L bag (White) 200L,NaN,Each,Each,NaN,200L,14,10,False,"14,000","25,00","37,38",",500"
4,4,Void fill 100 L bag (White) 100L,NaN,Each,Each,NaN,100L,14,10,False,"14,000","12,50","18,69",",250"
5,5,Air cushion machine (Blue),NaN,Each,Each,NaN,NaN,20,1,False,"20,000","1899,00","2839,01","10,000"


Observamos la cantidad de filas y columnas presentes en el dataset

In [21]:
print("Número de filas o datos: {}".format(len(datos_dim_stock)))
print("Número de columnas o variables: {}".format(len(datos_dim_stock.columns)))

Número de filas o datos: 671
Número de columnas o variables: 14


Para hacer una exploración más exhaustiva, hicimos un pandas profiling de los datos

In [22]:
profiling = ProfileReport(datos_dim_stock)
profiling.to_file(output_file="pandas_profiling/dimension_stock_item_pandas_profiling.html")
profiling

Summarize dataset:   7%|▋         | 2/27 [00:00<00:02, 11.02it/s, Describe variable:df_index]      


ImportError: cannot import name 'ABCIndex' from 'pandas.core.dtypes.generic' (c:\Users\Alvaro\anaconda3\lib\site-packages\pandas\core\dtypes\generic.py)

Del anterior reporte podemos concluir y considerar:
- No encontamos filas duplicadas
- Encontramos 1118 celdas vacías que corresponden al 11.1% de los datos. Como el objetivo de este laboratorio no es crear un modelo de ML sino responder a preguntas del negocio sobre sus datos y operación, no podemos reemplazar estos valores vacíos por otros, por lo que los dejaremos como están.
- Observamos que la variable Brand tiene un valor constante en las celdas que tienen un valor asignado, pero además presenta un 90% de ausencias, por lo que no aporta información relevante y podemos considerar eliminarla.

## Preprocesamiento de los datos

### Variable Dimension City

Como vimos en la exploración, las variables row ID y City_Key tienen un valor distinto en cada fila. Eliminamos entonces la variable row ID para dejar City_Key como identificador único. Igualmente eliminamos las variables Country, Continent, Region y Subregion 

In [23]:
datos_dim_city = datos_dim_city.drop(['row ID', 'Country', "Continent", "Region", "Subregion"], axis=1)

In [24]:
datos_dim_city.head()

,City_Key,City,State_Province,Sales_Territory,Latest_Recorded_Population
1,1,Carrollton,New York,Mideast,0
2,2,Carrollton,Virginia,Southeast,4574
3,3,Carrollton,Illinois,Great Lakes,2484
4,4,Carrollton,Missouri,Plains,3784
5,5,Carrollton,Ohio,Great Lakes,3241


### Variable Dimension Customer

Como vimos en la exploración de la variable Category, ésta tiene un mismo valor en cada celda. Esto indica que no aporta información relevante y procedemos a eliminarla. Igualmente eliminamos la variable Bill_To_Customer para dejar la variable Buying_Group para aportar la misma información

In [25]:
datos_dim_cust = datos_dim_cust.drop(['Category', 'Bill_To_Customer'], axis=1)

In [26]:
datos_dim_cust.head()

,Customer_Key,Customer,Buying_Group,Primary_Contact,Postal_Code
1,1,Tailspin Toys (Head Office),Tailspin Toys,Waldemar Fisar,90410.0
2,2,Tailspin Toys (Sylvanite- MT),Tailspin Toys,Lorena Cindric,90216.0
3,3,Tailspin Toys (Peeples Valley- AZ),Tailspin Toys,Bhaargav Rambhatla,90205.0
4,4,Tailspin Toys (Medicine Lodge- KS),Tailspin Toys,Daniel Roman,90152.0
5,5,Tailspin Toys (Gasport- NY),Tailspin Toys,Johanna Huiting,90261.0


### Variable Dimension Date

Como vimos en la exploración de las variables Day_Number y Day_val aportan la misma información, por lo que eliminamos Day_val. Lo mismo sucede con Month_val y Short_Month, por lo que eliminamos Short_Month

In [27]:
datos_dim_date = datos_dim_date.drop(['Day_val', 'Short_Month'], axis=1)

In [29]:
datos_dim_date.head()

,Date_key,Day_Number,Month_val,Calendar_Month_Number,Calendar_Year,Fiscal_Month_Number,Fiscal_Year
0,2013-01-01,1,January,1,2013,3,2013
1,2013-01-02,2,January,1,2013,3,2013
2,2013-01-03,3,January,1,2013,3,2013
3,2013-01-04,4,January,1,2013,3,2013
4,2013-01-05,5,January,1,2013,3,2013


### Variable Dimension Stock Item

Como vimos en el análisis de la variable Brand, ésta no aporta información relevante, por lo que podemos eliminarla.

In [30]:
datos_dim_stock = datos_dim_stock.drop(['Brand'], axis=1)

In [31]:
datos_dim_stock.head()

,Stock_Item_Key,Stock_Item,Color,Selling_Package,Buying_Package,Size_val,Lead_Time_Days,Quantity_Per_Outer,Is_Chiller_Stock,Tax_Rate,Unit_Price,Recommended_Retail_Price,Typical_Weight_Per_Unit
1,1,Void fill 400 L bag (White) 400L,NaN,Each,Each,400L,14,10,False,"14,000","50,00","74,75","1,000"
2,2,Void fill 300 L bag (White) 300L,NaN,Each,Each,300L,14,10,False,"14,000","37,50","56,06",",750"
3,3,Void fill 200 L bag (White) 200L,NaN,Each,Each,200L,14,10,False,"14,000","25,00","37,38",",500"
4,4,Void fill 100 L bag (White) 100L,NaN,Each,Each,100L,14,10,False,"14,000","12,50","18,69",",250"
5,5,Air cushion machine (Blue),NaN,Each,Each,NaN,20,1,False,"20,000","1899,00","2839,01","10,000"
